In [ ]:
import os

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:
#  DELETE ALL GRAPHS, currently throws a spurious error


for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:
#  CREATE A GRAPH

my_graph=my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


In [ ]:
# ##################################################################

In [ ]:
#  IMPORT DATA INTO GRAPH USING CSV

l_inp="gs://katana-demo-datasets/csv-datasets/ldbc/headers/sf-0.003-bi-node_headers.txt"
l_enp="gs://katana-demo-datasets/csv-datasets/ldbc/headers/sf-0.003-bi-edge_headers.txt"
   #
l_dir="gs://katana-demo-datasets/csv-datasets/ldbc"

import_data.csv(
   my_graph,
   input_node_path    = l_inp,
   input_edge_path    = l_enp,
   input_dir          = l_dir,
      #
   data_delimiter     = "|",
   schema_delimiter   = "|",
   files_have_headers = True
   )

print("--")


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
my_graph.query("CALL graph.schema() RETURN *")

In [ ]:
# ##################################################################

In [ ]:

l_result = my_graph.query("""

   MATCH (n:Person) - [ r:IS_LOCATED_IN] - (p:Place)

   RETURN p.name, COUNT(*)
   
   ORDER BY count(*) DESC
   LIMIT 10

   """)

print(l_result)

In [ ]:

#  Works

l_result = my_graph.query("""

   MATCH (n:Person) - [r:IS_LOCATED_IN] - (p:Place)

   WHERE p.name CONTAINS "New_York_City"
   WITH collect(n.lastName) AS surname, p, n

   RETURN surname, p

   """)

print(l_result)

In [ ]:

#  Fails

l_result = my_graph.query("""

   MATCH (n:Person) - [r:IS_LOCATED_IN] - (p:Place)

   WITH collect(n.lastName) AS surname, p, n
   WHERE p.name CONTAINS "New_York_City"

   RETURN surname, p

   """)

print(l_result)



In [ ]:

#  Error from above-

# OperationError: query C1hykuC9S44vMirfuDnNz99PJHaNNzg8iHUJzDTFnDbC-5Rj6yXJV5whwtNsgt error: ibquery/src/OpGraph.cpp:56): 
# backtrace: (../../libquery/src/GroupByAggregateOperator.cpp:414): backtrace: (../../libquery/src/GroupByAggregateOperator.cpp:167): 
# backtrace: (../../libquery/src/expressions/Evaluate.cpp:1972): backtrace: (../../libquery/src/expressions/Evaluate.cpp:1429): 
# backtrace: (../../libquery/src/EvaluateSelection.cpp:1397): error selecting property 
# 'lastName' of 'n': property evaluation function may hang and may be incorrect without a collective operation (EvaluateSelection.cpp:174)


In [ ]:
#  Fails

l_result = my_graph.query("""

   MATCH (n:Person) - [r:IS_LOCATED_IN] - (p:Place)

   WITH *, collect(n.lastName) AS surname
   WHERE p.name CONTAINS "New_York_City"

   RETURN surname, p

   """)

print(l_result)

In [ ]:

#  Error from above-

# OperationError: query VtRQ6fJvGBpPdeUkmC8dGz1mENkkLYptoxu51FxvAHB-2M1ntt6dc1w6VNhJY error: ibquery/src/OpGraph.cpp:56): 
# backtrace: (../../libquery/src/GroupByAggregateOperator.cpp:414): backtrace: (../../libquery/src/GroupByAggregateOperator.cpp:167): 
# backtrace: (../../libquery/src/expressions/Evaluate.cpp:1972): backtrace: (../../libquery/src/expressions/Evaluate.cpp:1429): 
# backtrace: (../../libquery/src/EvaluateSelection.cpp:1397): error selecting property 'lastName' of 'n': property evaluation function may hang 
# and may be incorrect without a collective operation (EvaluateSelection.cpp:174)
